# Alphabet Classification using Artificial Neural Networks (ANN) with Sci-Keras
This notebook uses Sci-Keras instead of the deprecated KerasClassifier to build and optimize the model.

## Step 1: Data Exploration and Preprocessing

In [12]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset
data = pd.read_csv('Alphabets_data.csv')

# Display the first few rows
data.head()

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [13]:
# Summarize the dataset
print("Number of samples:", data.shape[0])
print("Number of features:", data.shape[1])
print("Class distribution:")
print(data['letter'].value_counts())

Number of samples: 20000
Number of features: 17
Class distribution:
letter
U    813
D    805
P    803
T    796
M    792
A    789
X    787
Y    786
N    783
Q    783
F    775
G    773
E    768
B    766
V    764
L    761
R    758
I    755
O    753
W    752
S    748
J    747
K    739
C    736
H    734
Z    734
Name: count, dtype: int64


### Data Preprocessing
- Convert categorical labels to numerical values.
- Normalize the feature columns.
- Split the dataset into training and test sets.

In [14]:
# Preprocessing the data
label_encoder = LabelEncoder()
data['letter'] = label_encoder.fit_transform(data['letter'])

# Separate features and labels
X = data.drop(columns=['letter'])
y = data['letter']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


## Step 2: ANN Model Implementation

In [15]:
# Import necessary libraries for ANN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build a basic ANN model with one hidden layer
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')  # Multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

ModuleNotFoundError: No module named 'tensorflow.keras'

## Step 3: Model Evaluation

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate classification report
from sklearn.metrics import classification_report
y_pred = model.predict(X_test).argmax(axis=1)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

## Step 4: Hyperparameter Tuning using Sci-Keras
We will tune the following hyperparameters:
- Number of hidden layers and neurons
- Activation functions
- Learning rate

In [ ]:
# Import necessary libraries for hyperparameter tuning using Sci-Keras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Define the function for creating the ANN model
def create_model(activation='relu', optimizer='adam'):
    model = Sequential([
        Dense(64, input_shape=(X_train.shape[1],), activation=activation),
        Dense(32, activation=activation),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model using Sci-Keras' KerasClassifier
model = KerasClassifier(model=create_model, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {
    'model__activation': ['relu', 'tanh'],
    'optimizer': ['adam', 'rmsprop'],
    'epochs': [10, 20],
    'batch_size': [10, 20]
}

# Grid search using Sci-Keras
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best hyperparameters and performance
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

## Additional Visualizations and Explanation
We will now visualize the class distribution and the model's performance over time.

In [ ]:
# Plot the distribution of the classes (letters)
plt.figure(figsize=(10,6))
sns.countplot(x=data['letter'], palette='Set2')
plt.title('Distribution of Alphabets in the Dataset')
plt.ylabel('Count')
plt.xlabel('Alphabet Class (Encoded)')
plt.show()

# Plot the training and validation accuracy over epochs
plt.figure(figsize=(10,6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot the training and validation loss over epochs
plt.figure(figsize=(10,6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

## Step 4: Hyperparameter Tuning using Keras Tuner
We will use Keras Tuner to tune the following hyperparameters:
- Number of hidden layers and neurons
- Activation functions
- Learning rate

In [ ]:
# Import Keras Tuner
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam

# Define the function for creating the ANN model
def build_model(hp):
    model = Sequential()
    
    # Tune the number of units in the first Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(Dense(units=hp_units, activation='relu', input_shape=(X_train.shape[1],)))
    
    # Add a second hidden layer
    model.add(Dense(units=hp_units, activation='relu'))
    
    # Output layer
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    
    # Tune the learning rate for Adam optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Initialize the Keras Tuner
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='ANN_Tuner')

# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best number of units: {best_hps.get('units')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")